GIMME RAAAAM!

In [ ]:
'''a = []
while(1):
    a.append("1")'''

Install needed packages

In [ ]:
#using Coreference #==2.1.3
!pip install spacy==2.1.3
!pip install transformers==2.2.2
!pip install neuralcoref

     |████████████████████████████████| 27.7MB 1.3MB/s 
     |████████████████████████████████| 3.2MB 51.8MB/s 
     |████████████████████████████████| 92kB 11.5MB/s 
     |████████████████████████████████| 2.1MB 48.6MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.3 which is incompatible.
  Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Found existing installation: preshed 3.0.2
    Uninstalling preshed-3.0.2:
      Successfully uninstalled preshed-3.0.2
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 389kB 3.3MB/s 
     |███████████████████████████

In [ ]:
!pip install bert-extractive-summarizer
#A close physical contact in the past 14 days with a confirmed case of COVID-19 infection. This contrasts with the MERS-CoV MoH guiding document [13] in which VT for ARI requires a score of ≥4 to place patients in isolation.

  Created wheel for bert-extractive-summarizer: filename=bert_extractive_summarizer-0.4.2-cp36-none-any.whl size=13711 sha256=9241fc5ccf53c2e22ba5e6f746b13bbbd72f0a728ec70fc5babaaeaaa84a5cf6
  Stored in directory: /root/.cache/pip/wheels/13/bc/30/654eb9e657177a56cba927c5a20b6cd01fb229b1ed2bf9b371
Successfully built bert-extractive-summarizer


Run the BERT summarizer using sciBERT model and vocabulary: we first load the pickle with covid articles.

In [ ]:
from summarizer import Summarizer
from summarizer.coreference_handler import CoreferenceHandler
from transformers import *
#import spacy.cli


import pickle

100%|██████████| 40155833/40155833 [00:00<00:00, 58494270.76B/s]


In [ ]:
import spacy
spacy.cli.download("en_core_web_sm")
nlp = spacy.load('en_core_web_sm')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


Load covid articles from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Run this if you want to load all covid article and generate the summary for every article

In [ ]:
covid_articles = {}
with open("/content/drive/My Drive/COVID19/Data/covid_articles_updated_clean.p", "rb") as articleFile:
  covid_articles = pickle.load(articleFile)

Run this if you already have the summary and the clusters and you want to obtain summaries for any cluster (such that to obtain the topic related to any cluster)

In [ ]:
clustering = {}
with open("/content/drive/My Drive/COVID19/Data/clustering.p", "rb") as articleFile:
  clustering = pickle.load(articleFile)

covid_articles = {}
for cluster in clustering:
  for subcluster in clustering[cluster]:
    text = ""
    for sentence in clustering[cluster][subcluster]:
      text += sentence[2] + ". "
    covid_articles[(cluster, subcluster)] = text

In [ ]:
display(len(covid_articles))

3155

In [ ]:
display((covid_articles))

In [ ]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
import tensorflow as tf
tf.random.set_seed(42)
import torch
torch.manual_seed(42)
import numpy as np
np.random.seed(42)

Summarize covid articles

In [ ]:
def summarize(text):
  handler = CoreferenceHandler(greedyness=.4)

  custom_config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased')
  custom_config.output_hidden_states=True
  custom_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
  custom_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=custom_config)

  model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer) #sentence_handler=handler
  result = model(text, ratio = 0.3, min_length = 10, max_length=200)
  full = ''.join(result)
  return full

In [ ]:
from tqdm import tqdm_notebook as tqdm

covid_articles_summary = {}
for article in tqdm(covid_articles):

  covid_articles_summary[article] =  summarize(covid_articles[article])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.6/dist-packages/summarizer/cluster_features.py:97: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)
/usr/local/lib/python3.6/dist-packages/summarizer/cluster_features.py:97: ConvergenceWarning: Number of distinct clusters (38) found smaller than n_clusters (44). Possibly due to duplicate points in X.
  model = self.__get_model(k).fit(self.features)


In [ ]:
display(covid_articles_summary)

In [ ]:
with open("/content/drive/My Drive/COVID19/Data/covid_articles_summary_updated_clean_03.p", "wb+") as sumFile:
  pickle.dump(covid_articles_summary, sumFile)

In [ ]:
with open("/content/drive/My Drive/COVID19/Data/covid_articles_summary_clusters.p", "wb+") as sumFile:
  pickle.dump(covid_articles_summary, sumFile)

In [ ]:
display(covid_articles_summary)

In [ ]:
prova = {}
with open("covid_articles_summary_clean_05.p", "rb") as sumFile:
  prova = pickle.load(sumFile)


In [ ]:
display(prova)

Debug code below. Do not run, it's useless

In [ ]:
handler = CoreferenceHandler(greedyness=.4)

body = '''
Middle East Respiratory Syndrome Coronavirus (MERS-CoV) has plagued the Middle East since it was first reported in 2012 [1].
As of January 2020, the World Health Organization (WHO) has been notified of 2494 laboratory-confirmed cases with 858 mortalities [2] , with the majority of cases reported from the Kingdom of Saudi Arabia (KSA), which-at the time of this writing-has a total of 2131 laboratory-confirmed cases with 827 deaths [3] , making the case fatality rate (CFR) 39%.
Currently, KSA continues to report MERS-CoV cases on weekly basis, but the overall number of nosocomial transmissions has significantly been reduced [4] [5] [6] [7].
Recently, at the end of December 2019, a cluster of pneumonia cases were reported from Wuhan city, Hubei Province, China, linked to a wet seafood market with a new coronavirus identified as the etiologic agent currently named Severe Acute Respiratory Syndrome (SARS-CoV-2) [8, 9].
To date, the virus has rapidly spread causing-at the time of this writing-a total of 50,272 laboratory-confirmed cases and 16 The Global Surveillance Interim guidance by WHO developed [11] the following case definition for suspect case: a patient fulfilling either criteria A or B as detailed in Table 2 .
In contrast, the Saudi Center for Disease Prevention and Control (SCDC) defined a suspected case as the following [12] :
A person with acute respiratory illness (ARI) (fever with cough and/or shortness of breath) AND any of the following: 1
A history of travel to China in the 14 days prior to symptom onset.
A close physical contact in the past 14 days with a confirmed case of COVID-19 infection.
As it relates to KSA and with the ongoing MERS-CoV community cases and hospital acquired infections, there will be a huge challenge for health care workers (HCWs) to combat both viruses, MERS-CoV and SARS-CoV-2.
The current MERS-CoV case definition by Saudi CDC is highlighted in Table 3 .
But with the lack of standardized and approved point of care testing for MERS-CoV, HCWs continue to be challenged with rapidly detecting and properly isolating patients with MERS-CoV, especially among patients with congestive heart failure (CHF) and chronic kidney disease (CKD) who present with unexplained deterioration of their chronic conditions.
This challenge will now be faced by the whole global health community dealing with COVID-19 since the disease tends to be more prevalent and more severe in patients with CHF or CKD.
Those patients can present with broad range of signs and symptoms and should now be tested for both MERS-CoV and SARS-CoV-2 simultaneously, and with the already wide international spread of SARS-CoV-2, the travel history to China in the last 14 days should be ignored.
The new COVID-19 guidance document [12] produced by Saudi CDC provides HCFs with a new visual triage (VT) for ARI that has a checklist for placing patients in isolation precautions with score ≥6, with the weight of the points to history of travel to China in the last 14 days prior to symptoms onset, which is given five points.
This contrasts with the MERS-CoV MoH guiding document [13] in which VT for ARI requires a score of ≥4 to place patients in isolation.
None of these VT scores have been validated to show effectiveness.
With the ongoing MERS-CoV circulation in animal reservoir (dromedary camels) in the Arabian Peninsula and the continuing zoonotic spillover with 70% of cases resulting from nosocomial transmission [14] , the risk of emergence of COVID-19 within the same community would be overwhelming to the health-care infrastructure of the countries affected by MERS-CoV.
This is a true concern with the already rapid international spread to 25 countries outside China, especially with the ability of SARS-CoV-2 to spread from a totally asymptomatic person [15].
Since the declaration of PHEIC by WHO on January 31, 2020 [16] , 16 cities in mainland China with a population of around 60 million people remain under quarantine [17].
Many countries including Japan, the United States, France, Australia, Sri Lanka, Germany, Saudi Arabia, Russia, and Thailand among others evacuated their citizens through chartered flights [18] , and several countries have recommended travelers to avoid nonessential travel to China and cancelled scheduled flight in and out of China [19].
Many countries [12] enforced public health measures at ports of entry with trained staff, appropriate space, and equipment and supplies for managing potential ill travelers, implementing entry screening including temperature recording for travelers arriving from China (directly or indirectly), and mandatory declaration of recent travel to china to immigration and passport officers.
Although these measures may prevent the entry of active COVID-19 cases, history taught us that this system is not fully effective in preventing virus introduction into a country.
As per Saudi CDC, other public health measures include obligatory reporting of HCFs by calling a specific number and through the Health Electronic Surveillance Network.
his would activate a rapid response team tasked with completing epidemiological investigation in HCFs and community spontaneously, categorizing household and community contacts as per their symptoms, which, as with MERS-CoV, underplays the need for more liberal testing or isolating asymptomatic contacts [20].
A comparative analysis of factors influencing MERS-CoV outbreak in KSA compared to South Korea identified several exacerbating factors: which includes poor access to health-care professionals at general practitioner level, which can result in delays between disease development, progression, and diagnosis, individuals who relied on camels for daily living denied the relation between camels and MERS-CoV: kicking off the "Kiss Your Camel campaign, " which became increasingly popular in Social Media in 2015, and 75% of HCWs were unaware of how disease spreads, with a general feeling of workers and citizens is largely that MERS was not a substantial problem [21].
Similarly, the current COVID-19 outbreak and response have been accompanied by a massive "infodemic"-an overabundance of information-some accurate and many not-that makes it hard for people to find trustworthy sources and reliable guidance [10] , as most turn to social media, it is vital to track and respond to myths and rumors, an up-to-date reference should be maintained.
'''

body = body.replace("\n", " ")

custom_config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
custom_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=custom_config)

model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer) #sentence_handler=handler
result = model(body, ratio = 0.2, max_length=200)
full = ''.join(result)
print(result)

A close physical contact in the past 14 days with a confirmed case of COVID-19 infection. This contrasts with the MERS-CoV MoH guiding document [13] in which VT for ARI requires a score of ≥4 to place patients in isolation.
